# Setup

In [49]:
import pandas as pd
from datetime import timedelta
import os

In [50]:
#connect with drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
%cd /content/drive/MyDrive/GEE_temperature_TOP10
!ls

/content/drive/MyDrive/GEE_temperature_TOP10
convert_to_weekly.ipynb  temperature_23001.csv	temperature_68001.csv
temperature_05001.csv	 temperature_41001.csv	temperature_73001.csv
temperature_05360.csv	 temperature_50001.csv	temperature_76001.csv
temperature_08001.csv	 temperature_54001.csv	Weekly_files


In [52]:
out_dir = 'Weekly_files'

# Files

In [53]:
files = []

for file in os.listdir():
  if file.endswith('.csv'):
    files.append(file)
print(f'The number of files is: {len(files)}')
print(files)

The number of files is: 10
['temperature_76001.csv', 'temperature_54001.csv', 'temperature_50001.csv', 'temperature_73001.csv', 'temperature_68001.csv', 'temperature_41001.csv', 'temperature_23001.csv', 'temperature_05001.csv', 'temperature_05360.csv', 'temperature_08001.csv']


# Functions

In [54]:
def read_csv(file):
  df = pd.read_csv(file)
  df['date'] = pd.to_datetime(df['date'])
  return df

In [55]:
def fill_na(df):
  df_aux = pd.DataFrame()

  ### Fill NaN with last value
  df_aux['ffill'] = df.value.fillna(method="ffill")
  # Fill the NaN of firsth row
  df_aux['ffill'] = df_aux['ffill'].fillna(method="backfill")

  ### Fill NaN with the next value
  df_aux['backfill'] = df.value.fillna(method="backfill")
  # Fill the NaN of last row
  df_aux['backfill'] = df_aux['backfill'].fillna(method="ffill")
  
  # Create a new value column with NaN as mean of last and next temperature
  df['value'] = (df_aux['backfill'] + df_aux['ffill'])/2

  return df

In [56]:
def weekly_df(df):
  df_aux = df.resample('W', label='left', closed = 'left', on='date').mean()
  df_aux['LastDayWeek'] = df_aux.index
  df_aux['LastDayWeek'] = df_aux['LastDayWeek'] + timedelta(days=6)
  return df_aux

In [67]:
def file_to_csv(out_dir, filename, df):
  df.to_csv(f'{out_dir}/{filename}', index=False)
  print(f'File: {filename} saved in {out_dir}/')

### Extract weekly files as csv

In [68]:
for file in files:
  # Read file
  df = read_csv(file)
  # Fill NaN
  df = fill_na(df)
  # Convert to weekly
  df = weekly_df(df)
  # Convert to csv
  file_to_csv(out_dir, file, df)

File: temperature_76001.csv saved in Weekly_files/
File: temperature_54001.csv saved in Weekly_files/
File: temperature_50001.csv saved in Weekly_files/
File: temperature_73001.csv saved in Weekly_files/
File: temperature_68001.csv saved in Weekly_files/
File: temperature_41001.csv saved in Weekly_files/
File: temperature_23001.csv saved in Weekly_files/
File: temperature_05001.csv saved in Weekly_files/
File: temperature_05360.csv saved in Weekly_files/
File: temperature_08001.csv saved in Weekly_files/
